In [2]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.classification
import os
import featuretools as ft


In [15]:
def run_as(df, target_ft):
    try:
        os.remove('/tmp/autosklearn_regression_example_tmp')
    except:
        pass
    X = df.drop(columns=target_ft)
    y = df[target_ft]
    X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=120,
        per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
        output_folder='./tmp/autosklearn_regression_example_out',
    )
    automl.fit(X_train, y_train)

In [4]:
def gen_feats(df):
    es = ft.EntitySet(id = 'df')
    es.entity_from_dataframe(entity_id = 'data', dataframe = df, 
                         make_index = True, index = 'index')
    feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])
    return feature_matrix

In [5]:
df = pd.read_csv("blood.csv")
target_ft = "class"

In [6]:
#run_as(df,target_ft)
gen_feats(df)

,V1,V2,V3,V4,class,V2 + V3,V1 + V3,V3 + V4,class + V4,class + V2,...,class + V3 * V2,class + V3 * V1,V1 + V2 * V1 + V3,V1 * V2 + V4,V1 * V1 + V4,V1 + V3 * V3 + V4,class + V2 * V3,class + V1 * V3,V1 + V4 * V4,class + V4 * V3 + V4
index,,,,,,,,,,,,,,,,,,,,,
0,2,50,12500,98,2,12550,12502,12598,100,52,...,625100,25004,650104,296,200,157500196,650000,50000,9800,1259800
1,0,13,3250,28,2,3263,3250,3278,30,15,...,42276,0,42250,0,0,10653500,48750,6500,784,98340
2,1,16,4000,35,2,4016,4001,4035,37,18,...,64032,4002,68017,51,36,16144035,72000,12000,1260,149295
3,2,20,5000,45,2,5020,5002,5045,47,22,...,100040,10004,110044,130,94,25235090,110000,20000,2115,237115
4,1,24,6000,77,1,6024,6001,6077,78,25,...,144024,6001,150025,101,78,36468077,150000,12000,6006,474006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,23,2,500,38,1,502,523,538,39,3,...,1002,11523,13075,920,1403,281374,1500,12000,2318,20982
744,21,2,500,52,1,502,521,552,53,3,...,1002,10521,11983,1134,1533,287592,1500,11000,3796,29256
745,23,3,750,62,1,753,773,812,63,4,...,2253,17273,20098,1495,1955,627676,3000,18000,5270,51156
